In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

In [5]:
shots_spot_train = pd.read_csv('./assets/shots_spot_train.csv', index_col='Unnamed: 0')

In [19]:
shots_spot_train.fillna(0, inplace=True)

In [25]:
X = shots_spot_train.drop(['type_multiple_gunshots', 'type_single_gunshot'], axis=1)
Y = shots_spot_train[['type_multiple_gunshots', 'type_single_gunshot']]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, stratify=Y)

In [27]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20577, 17), (6859, 17), (20577, 2), (6859, 2))

In [28]:
ss  = StandardScaler()
rfc = RandomForestClassifier()

In [30]:
X_train_s = ss.fit_transform(X_train)
X_test_s  = ss.transform(X_test)

In [31]:
rfc.fit(X_train_s, y_train)
rfc.score(X_train_s, y_train)

0.6221023472809447

In [32]:
rfc.score(X_test_s, y_test)

0.4688730135588278